In [6]:
import socket
import time
%run "receive_DTO.ipynb"
%run "send_DTO.ipynb"
%run "game_settings.ipynb"
%run "bot_input.ipynb"

#### Socket

In [7]:
FORMAT = 'utf-8 '
PORT_PLAYER_1 = 8081
PORT_PLAYER_2 = 8082
SERVER = '127.0.0.1'

In [8]:

class client_socket:
    def __init__(self,server_ip,server_port):
        self.server_ip = server_ip
        self.server_port = server_port
        self.addr = (server_ip,server_port)
        self.start_connection()
        
    def send(self,msg):
        msg = bytes(msg,FORMAT)
        self.client.send(msg)
        return

    def receive(self):
        for game in range(0,AMOUNT_OF_GAMES):
            self.start_playing(game)
        self.client.close()
        
    def start_playing(self,game):
        botMsg = bot_input(self.dto)
        self.send(botMsg)
        turn = 2
        while(turn < RESTART_TURN + 1):
            if (PLAYER_NUM != 2 or turn != RESTART_TURN):
                self.get_DTO_message()
                
            is_last_game = game == AMOUNT_OF_GAMES - 1
            if (RESTART and turn == RESTART_TURN and is_last_game==False):
                    if (PLAYER_NUM == 1):
                        self.send("restart")
                    print('I WENT TO RESTART')
                    self.attempt_reconnect()
            else:
                botMsg = bot_input(self.dto)
                self.send(botMsg)
            turn+=1
          
    def get_DTO_message(self):
        try:
            msg = self.client.recv(8)
            msg = msg.decode(FORMAT)
            msg = self.client.recv(int(msg))
            self.dto = DTO.from_json(msg)
        except:
            print('AAAAAAAAAAAA' + msg)
            time.sleep(0.2)
            self.get_DTO_message()
        return self.dto
                
    def get_first_dto(self):
        try:
            msg = self.client.recv(8)
            msg = msg.decode(FORMAT)
            msg = self.client.recv(int(msg))
            self.dto = DTO.from_json(msg)
        except:
            self.set_team_name()
                
    def set_team_name(self):
        self.send(TEAM_NAME)
        self.get_first_dto()
        
    def start_connection(self):
        time.sleep(1)
        print('I was here')
        try:
            self.client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            self.client.connect(self.addr)
            print('I was here 2')
            self.set_team_name()
            print('I was here 3')
        except:
            self.start_connection()
        
    def attempt_reconnect(self):
        self.client.close()
        if(PLAYER_NUM == 1):
            time.sleep(1)
        else:
            time.sleep(2)
        self.start_connection()

In [9]:
def selectPlayer():
    if(PLAYER_NUM == 1):
        return client_socket(SERVER,PORT_PLAYER_1)
    else:
        return client_socket(SERVER,PORT_PLAYER_2)

In [10]:
client = selectPlayer()
client.receive()

I was here
I was here
I was here 2
I was here
I was here


KeyboardInterrupt: 